In [1]:
# You might need to install spacy and download an english model
# !pip install spacy
# !python -m spacy download en

In [27]:
!pip install sklearn

In [1]:
# We have a bunch of emails on disk

import glob
import io
import spacy
from collections import Counter
import pandas as pd
import email
import re

# nlp = spacy.load('en')

# corpus = Counter()
# emails = []

# with open('./data/SPAMTrain.label', 'r') as file:
#     for i, line in enumerate(file):
#         # You can always look at all of the files too!
#         print('.', end='')
        
#         spam, eml = line.split(' ')
#         eml = "./data/TRAINING/{}".format(eml.strip())
        
#         c = Counter()
#         with open(eml, 'r', errors='replace') as full_message: 
#             msg = email.message_from_string(full_message.read())
#             if not msg.is_multipart() and msg.get_content_type() == 'text/plain':
#                 payload = str(msg.get_payload())
#                 for t in nlp(re.sub(r'\s+', ' ', payload)):
#                     c[t.lemma_] += 1

#         if len(c) > 0:
#             emails.append({'SPAM_VALUE': spam, **dict(c)})

# eml_df = pd.DataFrame(emails)

# keys = []

# for c in eml_df.columns:
#     if eml_df[c].sum() > 10:
#         print('.', end='')
#         keys.append(c)

# eml_df = eml_df[keys]

# eml_df.to_csv('./email-training.csv')

In [2]:
eml_df = pd.read_csv('./email-training.csv')

eml_df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', ' ', '!', '"', '#', '$', '%', '&', ''',
       ...
       '~', '~$', '~/.kde', '~~~', '»', 'й', 'չ', '─', '│', '�'],
      dtype='object', length=5062)

In [4]:
eml_df['the']

0         NaN
1         NaN
2         3.0
3         6.0
4         8.0
5         1.0
6         3.0
7        11.0
8        12.0
9         1.0
10        4.0
11        1.0
12        NaN
13        9.0
14       46.0
15        1.0
16        NaN
17        4.0
18        NaN
19        3.0
20        8.0
21        4.0
22        1.0
23        1.0
24        8.0
25        7.0
26       11.0
27       91.0
28        NaN
29       20.0
        ...  
2911      NaN
2912     18.0
2913      3.0
2914     15.0
2915      3.0
2916      NaN
2917      6.0
2918     12.0
2919      7.0
2920     10.0
2921      NaN
2922      NaN
2923      6.0
2924      7.0
2925      5.0
2926      4.0
2927     29.0
2928      NaN
2929      8.0
2930      2.0
2931     19.0
2932      7.0
2933      NaN
2934      1.0
2935     17.0
2936    105.0
2937      4.0
2938     16.0
2939     13.0
2940      NaN
Name: the, Length: 2941, dtype: float64

In [14]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import classification_report

# Normal distribution bell curve: Gaussian
# Beroulli: 

clf = BernoulliNB(alpha=0.0)

X = eml_df[eml_df.columns[~eml_df.columns.isin(['SPAM_VALUE'])]].fillna(value=0)
y = eml_df['SPAM_VALUE']

clf.fit(X, y)

# Why is the value what it is? What would you do to fix this?

print(classification_report(clf.predict(X), y))

/home/hexgnu/.pyenv/versions/3.7.3/lib/python3.7/site-packages/sklearn/naive_bayes.py:485: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


              precision    recall  f1-score   support

           0       0.89      1.00      0.94       330
           1       1.00      0.98      0.99      2611

    accuracy                           0.99      2941
   macro avg       0.94      0.99      0.97      2941
weighted avg       0.99      0.99      0.99      2941



In [15]:
eml_df.groupby('SPAM_VALUE').size()

SPAM_VALUE
0     307
1    2034
dtype: int64

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = BernoulliNB(alpha=0.06)
# clf = GaussianNB()

clf.fit(X_train, y_train)

print(classification_report(clf.predict(X_test), y_test))

              precision    recall  f1-score   support

           0       0.85      0.93      0.89        59
           1       0.99      0.98      0.99       530

    accuracy                           0.98       589
   macro avg       0.92      0.96      0.94       589
weighted avg       0.98      0.98      0.98       589



In [6]:
from sklearn.metrics import confusion_matrix

confusion_matrix(clf.predict(X_test), y_test)

array([[ 61,  35],
       [  4, 489]])

I was able to get ~0.94 for precision on both Ham(1) and Spam(0). That's pretty good! The recall isn't so good actually. A 0.60 recall isn't so great, although we could probably do better on parsing the Emails themselves. Think of some reasons how this result could get better.

# BONUS word2vec

Word2Vec is an amazing model to fit natural language to a vectorized representation. Matter of fact it's called representation learning and basically will take text and output something that we can do math on!

Below I've done the heavy lifting of giving you a word2vec file to train on. Try out GaussianNB and see what happens. Is it any better or worse?

In [22]:
# emails = []
# import numpy as np

# with open('./data/SPAMTrain.label', 'r') as file:
#     for i, line in enumerate(file):
#         # You can always look at all of the files too!
#         print('.', end='')
        
#         spam, eml = line.split(' ')
#         eml = "./data/TRAINING/{}".format(eml.strip())
        
#         vector = np.zeros(0)
#         with open(eml, 'r', errors='replace') as full_message: 
#             msg = email.message_from_string(full_message.read())
#             if not msg.is_multipart() and msg.get_content_type() == 'text/plain':
#                 payload = str(msg.get_payload())
                
#                 s = nlp(re.sub(r'\s+', ' ', payload))
                
#                 vector = s.vector

#         if np.sum(vector) > 0:
#             emails.append({'SPAM_VALUE': spam, **dict(zip(range(384), vector))})

# eml_df = pd.DataFrame(emails)

# eml_df.to_csv('./emails-word2vec.csv')

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [6]:
eml_df = pd.read_csv('./emails-word2vec.csv')

eml_df.head()

,Unnamed: 0,SPAM_VALUE,0,1,2,3,4,5,6,7,...,374,375,376,377,378,379,380,381,382,383
0,0,0,0.248722,1.391528,1.087005,1.136443,0.088674,1.147383,-1.577564,-0.270032,...,-0.013487,-0.028195,0.032172,0.031236,0.210149,0.154497,-0.072023,0.115364,0.265590,-0.013191
1,1,0,0.576246,0.697309,1.015784,1.422330,0.060741,-0.074634,-0.671568,-0.578649,...,0.038255,0.004724,-0.024387,-0.000826,0.011039,0.079151,-0.082945,0.182521,-0.004091,-0.058852
2,2,1,-0.172061,0.395070,0.754064,0.677915,0.453942,-0.450965,-0.441118,-0.652330,...,0.109467,-0.023011,0.020112,-0.051621,0.087226,0.066990,-0.057918,0.115030,0.127236,0.021688
3,3,1,-0.050888,0.085505,0.706955,1.214564,0.098087,-0.053080,-0.548602,0.080831,...,0.173510,0.044773,0.093819,-0.079164,0.136898,0.110524,-0.019751,0.160244,0.141444,-0.014389
4,4,1,0.647508,0.685397,0.534541,1.115465,0.333897,-0.481878,-1.361484,0.132314,...,0.265855,0.035184,0.228359,0.056503,0.240884,0.111331,-0.003491,0.142417,0.179619,0.119642


In [34]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

clf = LinearSVC(C=50)

X = eml_df[eml_df.columns[~eml_df.columns.isin(['SPAM_VALUE'])]].fillna(value=0)
y = eml_df['SPAM_VALUE']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf.fit(X_train, y_train)

print(classification_report(clf.predict(X_test), y_test))

             precision    recall  f1-score   support

          0       0.69      0.70      0.70        54
          1       0.96      0.96      0.96       415

avg / total       0.93      0.93      0.93       469

